<a href="https://colab.research.google.com/github/IvchenkoIO/ML_Cyber/blob/main/training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [157]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Lambda, Activation, concatenate
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
import tensorflow_probability as tfp
from jinja2 import Template
import matplotlib.pyplot as plt
import os

tfd = tfp.distributions

class MDNRNNModel:
    def __init__(self, input_shape, output_shape, num_mixtures, lstm_units, learning_rate=0.001):
        self.input_shape = input_shape
        self.output_shape = output_shape
        self.num_mixtures = num_mixtures
        self.lstm_units = lstm_units
        self.learning_rate = learning_rate
        self.model = self.build_model()
        self.model.compile(optimizer=Adam(learning_rate=self.learning_rate), loss=self.get_mixture_loss())

    def get_mixture_loss(self):
        def mixture_loss(y_true, y_pred):
            y_true = K.reshape(y_true, [-1, self.input_shape[0], self.output_shape])
            y_true = K.expand_dims(y_true, axis=-1)  # Shape: (batch_size, sequence_length, num_features, 1)
            y_true = K.tile(y_true, [1, 1, 1, self.num_mixtures])  # Shape: (batch_size, sequence_length, num_features, num_mixtures)

            # Extracting pi, mu, and sigma
            pi = y_pred[:, :, :self.num_mixtures * self.output_shape]
            mu = y_pred[:, :, self.num_mixtures * self.output_shape:2*self.num_mixtures*self.output_shape]
            sigma = y_pred[:, :, 2*self.num_mixtures*self.output_shape:]

            pi = K.reshape(pi, (-1, self.input_shape[0], self.output_shape, self.num_mixtures))
            mu = K.reshape(mu, (-1, self.input_shape[0], self.output_shape, self.num_mixtures))
            sigma = K.reshape(sigma, (-1, self.input_shape[0], self.output_shape, self.num_mixtures))

            pi = K.softmax(pi, axis=-1)
            sigma = K.exp(sigma)

            dist = tfp.distributions.Normal(loc=mu, scale=sigma)
            prob = dist.prob(y_true)
            prob = K.sum(pi * prob, axis=-1)
            loss = -K.log(prob + 1e-8)
            return K.mean(loss)
        return mixture_loss

    def build_model(self):
        inputs = Input(shape=self.input_shape, name='Input_Layer')
        lstm_1 = LSTM(self.lstm_units, return_sequences=True, name='LSTM_Layer_1')(inputs)
        lstm_2 = LSTM(self.lstm_units, return_sequences=True, name='LSTM_Layer_2')(lstm_1)
        lstm_3 = LSTM(self.lstm_units, return_sequences=True, name='LSTM_Layer_3')(lstm_2)

        mdn_output = Dense(self.num_mixtures * 3 * self.output_shape, name='MDN_Output')(lstm_3)

        pi = Lambda(lambda x: x[:, :, :self.num_mixtures * self.output_shape], name='Pi_Lambda')(mdn_output)
        mu = Lambda(lambda x: x[:, :, self.num_mixtures * self.output_shape:2*self.num_mixtures*self.output_shape], name='Mu_Lambda')(mdn_output)
        sigma = Lambda(lambda x: x[:, :, 2*self.num_mixtures*self.output_shape:], name='Sigma_Lambda')(mdn_output)

        pi = Activation('softmax', name='Pi_Activation')(pi)
        sigma = Activation('exponential', name='Sigma_Activation')(sigma)

        output = concatenate([pi, mu, sigma], name='Output_Concatenate')
        return Model(inputs=inputs, outputs=output, name='MDN_RNN_Model')

    def train(self, x_train, y_train, epochs=50, batch_size=64):
        self.model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size)

    def predict(self, x_test):
        return self.model.predict(x_test)

    def print_data_shapes(self, x_data, y_data, data_name=""):
        print(f"{data_name} Input Data Shape: {x_data.shape}\n{x_data}")
        if y_data is not None:
            print(f"{data_name} Output Data Shape: {y_data.shape}\n{y_data}")
        else:
            print(f"{data_name} Output Data is None")

    def analyze_predictions(self, predictions, y_test):
      analyses = []
      batch_size, sequence_length, _ = predictions.shape

      for i in range(batch_size):
          for t in range(sequence_length):
              pi = predictions[i, t, :self.num_mixtures * self.output_shape]
              mu = predictions[i, t, self.num_mixtures * self.output_shape:2 * self.num_mixtures * self.output_shape]
              sigma = predictions[i, t, 2 * self.num_mixtures * self.output_shape:]

              pi = pi.reshape(self.output_shape, self.num_mixtures)
              mu = mu.reshape(self.output_shape, self.num_mixtures)
              sigma = sigma.reshape(self.output_shape, self.num_mixtures)

              predicted_values = np.sum(pi * mu, axis=-1)
              actual_values = y_test[i, t, :]
              errors = actual_values - predicted_values
              abs_errors = np.abs(errors)

              analysis = {
                  'index': (i, t),
                  'predicted_values': predicted_values.tolist(),
                  'actual_values': actual_values.tolist(),
                  'errors': errors.tolist(),
                  'abs_errors': abs_errors.tolist(),
                  'pi': pi.tolist(),
                  'mu': mu.tolist(),
                  'sigma': sigma.tolist(),
                  'anomalies': (abs_errors > np.mean(abs_errors) + 2 * np.std(abs_errors)).tolist()  # Example threshold
              }
              analyses.append(analysis)

      mean_abs_error = np.mean([np.mean(a['abs_errors']) for a in analyses])
      max_abs_error = np.max([np.max(a['abs_errors']) for a in analyses])
      min_abs_error = np.min([np.min(a['abs_errors']) for a in analyses])

      summary = {
          'mean_abs_error': mean_abs_error,
          'max_abs_error': max_abs_error,
          'min_abs_error': min_abs_error
      }

      return analyses, summary


    def save_predictions_to_html(self, x_test, y_test, predictions, analyses, summary, file_name='predictions.html'):
      if not os.path.exists('plots'):
          os.makedirs('plots')

      # Iterate through each batch and sequence within the batch
      for i in range(predictions.shape[0]):  # iterate through batches
          for t in range(predictions.shape[1]):  # iterate through sequences in each batch
              plt.figure(figsize=(10, 6))
              pi = predictions[i, t, :self.num_mixtures * self.output_shape]
              mu = predictions[i, t, self.num_mixtures * self.output_shape:2 * self.num_mixtures * self.output_shape]
              sigma = predictions[i, t, 2 * self.num_mixtures * self.output_shape:]

              pi = pi.reshape(self.output_shape, self.num_mixtures)
              mu = mu.reshape(self.output_shape, self.num_mixtures)
              sigma = sigma.reshape(self.output_shape, self.num_mixtures)

              for j in range(self.num_mixtures):
                  plt.plot(mu[:, j], pi[:, j], 'o')
                  plt.errorbar(mu[:, j], pi[:, j], xerr=sigma[:, j], fmt='o')
              plt.title(f'Mixture Components for Prediction {i}_{t}')
              plt.xlabel('Mu')
              plt.ylabel('Pi')
              plt.savefig(f'plots/prediction_{i}_{t}.png')
              plt.close()

      template = Template("""
          <html>
          <head>
              <title>MDN-RNN Predictions - Nurlybek Maghzan</title>
              <style>
                  body {
                      font-family: Arial, sans-serif;
                      margin: 20px;
                  }
                  h1, h2, h3 {
                      color: #2c3e50;
                  }
                  table {
                      width: 100%;
                      border-collapse: collapse;
                      margin-bottom: 20px;
                  }
                  table, th, td {
                      border: 1px solid #ddd;
                  }
                  th, td {
                      padding: 15px;
                      text-align: left;
                  }
                  tr:nth-child(even) {
                      background-color: #f9f9f9;
                  }
                  th {
                      background-color: #2c3e50;
                      color: white;
                  }
                  summary {
                      cursor: pointer;
                      font-weight: bold;
                      background-color: #ecf0f1;
                      padding: 10px;
                      border: 1px solid #bdc3c7;
                      border-radius: 5px;
                  }
                  details[open] summary {
                      background-color: #bdc3c7;
                  }
                  details > div {
                      padding: 10px;
                      border: 1px solid #bdc3c7;
                      border-radius: 5px;
                      margin-bottom: 10px;
                  }
                  .section {
                      margin-bottom: 30px;
                  }
                  .section p {
                      margin: 10px 0;
                  }
                  .anomaly {
                      background-color: #ffcccc;
                  }
              </style>
          </head>
          <body>
              <h1>MDN-RNN Predictions - <i>Nurlybek Maghzan</i> </h1>

              <p>Этот отчет содержит анализ предсказаний модели MDN-RNN. В нем представлены входные данные, предсказания модели, выходные данные и сводка анализа.</p>

              <h2>Введение</h2>
              <p>MDN-RNN (Mixture Density Network - Recurrent Neural Network) - это модель, которая сочетает в себе рекуррентные нейронные сети (RNN) с сетями смеси плотностей (MDN). Такая комбинация позволяет прогнозировать временные ряды и неопределенные данные, что особенно полезно в задачах кибербезопасности для оценки вероятности различных инцидентов и разработки стратегий защиты.</p>
              <p>В данной работе использовалась модель MDN-RNN для прогнозирования инцидентов кибербезопасности в системе управления железнодорожными перевозками. Модель анализирует временные и числовые характеристики данных и подбирает оптимальные стратегии защиты информации.</p>

              <h2>Цель</h2>
              <p>Целью данного проекта было разработать и обучить модель MDN-RNN для прогнозирования инцидентов кибербезопасности и оценки оптимальных стратегий защиты информации. Входные данные представляют собой временные ряды характеристик системы, а выходные данные - вероятности, средние значения и стандартные отклонения для различных сценариев инцидентов.</p>

              <details open class="section">
                  <summary>Input Data</summary>
                  <div>
                      <p>Входные данные представляют собой последовательности характеристик, которые используются моделью для предсказания.</p>
                      <table>
                          <tr>
                              <th>Index</th>
                              <th>Values</th>
                          </tr>
                          {% for i in range(x_test.shape[0]) %}
                              {% for t in range(x_test.shape[1]) %}
                                  <tr>
                                      <td>{{ i }}_{{ t }}</td>
                                      <td>{{ x_test[i, t] }}</td>
                                  </tr>
                              {% endfor %}
                          {% endfor %}
                      </table>
                  </div>
              </details>

              <details class="section">
                  <summary>Predictions</summary>
                  <div>
                      <p>В этом разделе представлены предсказания модели, включая вероятности компонентов (Pi), средние значения (Mu) и стандартные отклонения (Sigma).</p>
                      {% for analysis in analyses %}
                          <details class="section">
                              <summary>Prediction {{ analysis.index }}</summary>
                              <div>
                                  <img src="plots/prediction_{{ analysis.index[0] }}_{{ analysis.index[1] }}.png" alt="Prediction {{ analysis.index }}">
                                  <table>
                                      <tr>
                                          <th>Component</th>
                                          <th>Pi</th>
                                          <th>Mu</th>
                                          <th>Sigma</th>
                                      </tr>
                                      {% for j in range(num_mixtures) %}
                                          <tr class="{{ 'anomaly' if analysis.anomalies[j] else '' }}">
                                              <td>{{ j }}</td>
                                              <td>{{ analysis.pi[j] }}</td>
                                              <td>{{ analysis.mu[j] }}</td>
                                              <td>{{ analysis.sigma[j] }}</td>
                                          </tr>
                                      {% endfor %}
                                  </table>
                                  <p><strong>Predicted Values:</strong> {{ analysis.predicted_values }}</p>
                                  <p><strong>Actual Values:</strong> {{ analysis.actual_values }}</p>
                                  <p><strong>Error:</strong> {{ analysis.errors }}</p>
                              </div>
                          </details>
                      {% endfor %}
                  </div>
              </details>

              <details class="section">
                  <summary>Output Data</summary>
                  <div>
                      <p>Выходные данные представляют собой фактические значения, которые модель пыталась предсказать.</p>
                      <table>
                          <tr>
                              <th>Index</th>
                              <th>Values</th>
                          </tr>
                          {% for i in range(y_test.shape[0]) %}
                              {% for t in range(y_test.shape[1]) %}
                                  <tr>
                                      <td>{{ i }}_{{ t }}</td>
                                      <td>{{ y_test[i, t] }}</td>
                                  </tr>
                              {% endfor %}
                          {% endfor %}
                      </table>
                  </div>
              </details>

              <details class="section">
                  <summary>Summary</summary>
                  <div>
                      <p>Этот раздел содержит сводку анализа предсказаний, включая среднюю абсолютную ошибку (MAE), максимальную и минимальную абсолютные ошибки.</p>
                      <p><strong>Mean Absolute Error:</strong> {{ summary.mean_abs_error }}</p>
                      <p><strong>Max Absolute Error:</strong> {{ summary.max_abs_error }}</p>
                      <p><strong>Min Absolute Error:</strong> {{ summary.min_abs_error }}</p>
                  </div>
              </details>

              <details class="section">
                  <summary>Conclusions</summary>
                  <div>
                      <p><strong>Выводы:</strong></p>
                      <p>Анализ предсказаний модели MDN-RNN показал следующие результаты:</p>
                      <ul>
                          <li><strong>Средняя абсолютная ошибка (MAE):</strong> {{ summary.mean_abs_error }} - это означает, что в среднем модель ошибается на это значение при предсказании значений.</li>
                          <li><strong>Максимальная абсолютная ошибка:</strong> {{ summary.max_abs_error }} - наибольшая ошибка предсказания, показывающая крайние случаи, когда модель значительно отклонилась от фактического значения.</li>
                          <li><strong>Минимальная абсолютная ошибка:</strong> {{ summary.min_abs_error }} - наименьшая ошибка предсказания, показывающая случаи, когда модель была наиболее точна.</li>
                      </ul>
                      <p>Эти метрики помогают оценить качество модели и понять, насколько хорошо она справляется с предсказанием данных. Средняя абсолютная ошибка (MAE) указывает на общую точность модели, а максимальная и минимальная ошибки помогают выявить случаи, когда модель была наиболее и наименее точной соответственно. На основе этих данных можно сделать вывод о том, насколько надежна модель и где возможны улучшения.</p>
                      <p>Прогнозирование инцидентов кибербезопасности с помощью MDN-RNN позволяет заранее оценивать вероятности различных угроз и разрабатывать стратегии для их предотвращения. Это помогает повысить безопасность информации и обеспечить устойчивость системы к потенциальным атакам.</p>
                  </div>
              </details>
          </body>
          </html>
      """)

      html_content = template.render(predictions=predictions, num_mixtures=self.num_mixtures, x_test=x_test, y_test=y_test, analyses=analyses, summary=summary)
      with open(file_name, 'w') as f:
          f.write(html_content)

# Parameters
# Create the model

from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/MyDrive/Study_DFs/chunk_1.csv')
df

#x_train = np.random.rand(1000, 20, 15)
#y_train = np.random.rand(1000, 1)
#x_test = np.random.rand(100, 20, 15)
#y_test = np.random.rand(100, 1)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,1578326400001,13.43.52.51,18.70.112.62,40,57354
0,1578326400005,16.79.101.100,12.48.65.39,92,11895
1,1578326400007,18.43.118.103,14.51.30.86,27,898
2,1578326400011,15.71.108.118,14.50.119.33,57,7496
3,1578326400012,14.33.30.103,15.24.31.23,115,20979
4,1578326400012,18.121.115.31,13.56.39.74,92,8620
...,...,...,...,...,...
1995,1578326402960,12.54.59.43,17.59.50.43,29,28770
1996,1578326402960,13.40.68.24,13.40.30.60,72,1205
1997,1578326402961,12.30.62.113,13.51.26.27,52,7440
1998,1578326402962,16.54.111.54,12.56.32.101,52,61024


In [158]:
column_mapping = {
    '1578326400001': 'time',
    '13.43.52.51': 'src',
    '18.70.112.62': 'dst',
    '40': 'port',
    '57354': 'byte_count'
}
#df
# Rename the columns using the rename method
df.rename(columns=column_mapping, inplace=True)
#df
def ip_to_int(ip):
    return int(''.join(['{:08b}'.format(int(x)) for x in ip.split('.')]), 2)
print("Renamed columns:", df.columns)


Renamed columns: Index(['time', 'src', 'dst', 'port', 'byte_count'], dtype='object')


In [159]:
df['src'] = df['src'].apply(ip_to_int)
df['dst'] = df['dst'].apply(ip_to_int)
df

,time,src,dst,port,byte_count
0,1578326400005,273638756,204488999,92,11895
1,1578326400007,304838247,238231126,27,898
2,1578326400011,256339062,238188321,57,7496
3,1578326400012,237051495,253239063,115,20979
4,1578326400012,309949215,221783882,92,8620
...,...,...,...,...,...
1995,1578326402960,204880683,289092139,29,28770
1996,1578326402960,220742680,220732988,72,1205
1997,1578326402961,203308657,221452827,52,7440
1998,1578326402962,272002870,205004901,52,61024


In [160]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df[['time', 'src', 'dst', 'port', 'byte_count']] = scaler.fit_transform(df[['time', 'src', 'dst', 'port', 'byte_count']])

In [161]:
df

,time,src,dst,port,byte_count
0,0.000000,0.568437,0.009575,0.686275,0.088735
1,0.000676,0.820584,0.282255,0.049020,0.006408
2,0.002028,0.428624,0.281909,0.343137,0.055803
3,0.002366,0.272746,0.403538,0.911765,0.156740
4,0.002366,0.861890,0.149340,0.686275,0.064217
...,...,...,...,...,...
1995,0.998986,0.012749,0.693278,0.068627,0.215065
1996,0.998986,0.140942,0.140847,0.490196,0.008707
1997,0.999324,0.000044,0.146664,0.294118,0.055383
1998,0.999662,0.555216,0.013744,0.294118,0.456527


In [162]:
# Create sequences
sequence_length = 3
sequences = []
targets = []

for i in range(len(df) - sequence_length):
    sequences.append(df.iloc[i:i+sequence_length].values)
    targets.append(df.iloc[i+1:i+sequence_length+1].values)

sequences = np.array(sequences)
targets = np.array(targets)

# Create batches
batch_size = 2
num_batches = len(sequences) // batch_size

batches = []
target_batches = []

for i in range(num_batches):
    batch = sequences[i * batch_size:(i + 1) * batch_size]
    target_batch = targets[i * batch_size:(i + 1) * batch_size]
    batches.append(batch)
    target_batches.append(target_batch)

batches = np.array(batches)
target_batches = np.array(target_batches)

# Output the prepared data shapes
print("Input batches shape:", batches.shape)
print("Target batches shape:", target_batches.shape)

# Example of accessing a batch
print("Example input batch:\n", batches[0])
print("Example target batch:\n", target_batches[0])

Input batches shape: (998, 2, 3, 5)
Target batches shape: (998, 2, 3, 5)
Example input batch:
 [[[0.00000000e+00 5.68436584e-01 9.57459472e-03 6.86274510e-01
   8.87346719e-02]
  [6.76155090e-04 8.20584031e-01 2.82254839e-01 4.90196078e-02
   6.40824088e-03]
  [2.02840567e-03 4.28624250e-01 2.81908919e-01 3.43137255e-01
   5.58026022e-02]]

 [[6.76155090e-04 8.20584031e-01 2.82254839e-01 4.90196078e-02
   6.40824088e-03]
  [2.02840567e-03 4.28624250e-01 2.81908919e-01 3.43137255e-01
   5.58026022e-02]
  [2.36648321e-03 2.72746362e-01 4.03538478e-01 9.11764706e-01
   1.56739882e-01]]]
Example target batch:
 [[[6.76155090e-04 8.20584031e-01 2.82254839e-01 4.90196078e-02
   6.40824088e-03]
  [2.02840567e-03 4.28624250e-01 2.81908919e-01 3.43137255e-01
   5.58026022e-02]
  [2.36648321e-03 2.72746362e-01 4.03538478e-01 9.11764706e-01
   1.56739882e-01]]

 [[2.02840567e-03 4.28624250e-01 2.81908919e-01 3.43137255e-01
   5.58026022e-02]
  [2.36648321e-03 2.72746362e-01 4.03538478e-01 9.117647

In [163]:
sequence_length = 3  # Each sequence has 3 time steps
num_features = 5  # Each time step has 5 features
input_shape = (sequence_length, num_features)
output_shape = num_features  # Predicting 5 features
num_mixtures = 4  # Number of Gaussian mixtures
lstm_units = 100  # Number of LSTM units
mdn_rnn = MDNRNNModel(input_shape, output_shape, num_mixtures, lstm_units)

In [164]:
x_train = batches.reshape(-1, sequence_length, num_features)
y_train = target_batches.reshape(-1, sequence_length, num_features)

# Training the model
mdn_rnn.train(x_train, y_train,epochs=10)

# Making predictions
predictions =mdn_rnn.predict(x_train[:1])  # Example prediction on the first sequence

print("Predictions:", predictions)

Epoch 1/10
32/32 [==============================] - 8s 20ms/step - loss: 1.7520
Epoch 2/10
32/32 [==============================] - 1s 22ms/step - loss: 1.1214
Epoch 3/10
32/32 [==============================] - 1s 21ms/step - loss: 0.9769
Epoch 4/10
32/32 [==============================] - 1s 21ms/step - loss: 0.9641
Epoch 5/10
32/32 [==============================] - 1s 23ms/step - loss: 0.9619
Epoch 6/10
32/32 [==============================] - 1s 23ms/step - loss: 0.9614
Epoch 7/10
32/32 [==============================] - 1s 21ms/step - loss: 0.9607
Epoch 8/10
32/32 [==============================] - 1s 22ms/step - loss: 0.9601
Epoch 9/10
32/32 [==============================] - 1s 22ms/step - loss: 0.9598
Epoch 10/10
1/1 [==============================] - 1s 1s/step
Predictions: [[[4.37991024e-04 2.85772898e-04 1.90636143e-01 8.96841346e-04
   1.54993101e-03 7.54118548e-04 2.03353632e-03 1.72952394e-04
   5.87343937e-04 7.18382653e-04 2.61217076e-02 2.61526904e-04
   2.49375385e-0

In [165]:
analyses, summary = mdn_rnn.analyze_predictions(predictions, y_train[:1])

# Print predictions
print(f"Predictions Shape: {predictions.shape}\n{predictions}")
output_file_path = "/content/predictions.html"
# Save predictions to HTML
#print(f"Predictions Shape: {predictions.shape}\n{predictions}")
#print(f"Analysis Summary: {summary}")
mdn_rnn.save_predictions_to_html(x_train, y_train, predictions, analyses, summary)


import os
if os.path.exists(output_file_path):
    print(f"File created successfully: {output_file_path}")
else:
    print("File not found.")

# Download the file
from google.colab import files
files.download(output_file_path)

Predictions Shape: (1, 3, 60)
[[[4.37991024e-04 2.85772898e-04 1.90636143e-01 8.96841346e-04
   1.54993101e-03 7.54118548e-04 2.03353632e-03 1.72952394e-04
   5.87343937e-04 7.18382653e-04 2.61217076e-02 2.61526904e-04
   2.49375385e-04 2.63647467e-04 7.66950727e-01 5.93162142e-04
   3.02639644e-04 5.84882591e-03 1.05145015e-03 2.83932168e-04
   4.59023088e-01 4.51904535e-01 4.48556513e-01 4.49583590e-01
   3.95716995e-01 4.00196314e-01 3.72043312e-01 3.82829070e-01
   2.78786033e-01 2.98795491e-01 2.89415359e-01 3.05832416e-01
   4.80652124e-01 4.82276380e-01 4.95452940e-01 4.87463474e-01
   1.29063010e-01 1.51592404e-01 1.41954154e-01 1.20495453e-01
   4.33957390e-03 4.04966762e-03 2.97010201e-03 3.91252851e-03
   2.73887371e-03 2.54737586e-03 2.18676520e-03 3.36521328e-03
   2.36268644e-03 2.60435045e-03 1.53849053e-03 2.86337198e-03
   4.11381340e-03 3.88867082e-03 1.50626712e-03 4.04102402e-03
   3.57419066e-03 1.68244867e-03 2.47774413e-03 3.41479154e-03]
  [1.19423321e-05 5.0871

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>